In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

from numpy import set_printoptions
from scipy.stats import skew
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import classification_report,confusion_matrix

import keras
#from keras.optimizers import SGD
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import LeaveOneOut

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

import xgboost
import lightgbm as lgb
from sklearn.metrics import roc_auc_score


import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dropout, Activation, Dense, BatchNormalization
#from keras.optimizers import RMSprop

import warnings
warnings.filterwarnings("ignore")

<ipython-input-2-46aa72f88981>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [ ]:
import os
from google.colab import drive
import zipfile

# This mounts your Google Drive to the Colab VM.
drive.mount('/content/drive')

# # Path to the kaggle.json file on your Google Drive
# kaggle_json_path = "/content/drive/MyDrive/Kaggle_Json/kaggle.json"

# # Check if the kaggle.json file exists in the folder
# if not os.path.isfile(kaggle_json_path):
#     print(f"Please upload the kaggle.json file to the folder: {kaggle_json_path}")
# else:
#     # Copy the kaggle.json file to the correct location and set permissions
#     os.makedirs('/root/.kaggle/', exist_ok=True)
#     os.system(f"cp {kaggle_json_path} /root/.kaggle/")
#     os.chmod("/root/.kaggle/kaggle.json", 600)
#     print("kaggle.json file uploaded successfully.")

#     # Kaggle dataset URL segment (this is usually the path that comes after kaggle.com/datasets/)
#     dataset_url = "nadyinky/sephora-products-and-skincare-reviews/versions/1"

#     # Make sure you have the Kaggle API installed
#     !pip install kaggle --quiet

#     # Download dataset and save it to your Google Drive
#     #os.makedirs("/content/drive/MyDrive/Skincare Recommendation Dataset", exist_ok=True)
#     os.system(f"kaggle datasets download -d {dataset_url} --path '/content/drive/MyDrive/Skincare Recommendation Dataset 2' --unzip")

#     print("Dataset downloaded and extracted to your Google Drive folder.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from google.colab import drive
# import zipfile
# import os

# def extract_zip_from_drive(zip_path):
#     """
#     Extracts a zip file from the specified path on Google Drive.

#     Parameters:
#     zip_path (str): The path to the zip file on Google Drive.

#     Returns:
#     None
#     """
#     # Mount Google Drive
#     #drive.mount('/content/drive')

#     # Ensure the zip file exists
#     if not os.path.exists(zip_path):
#         print(f"The zip file {zip_path} does not exist.")
#         return

#     # Define the folder to extract to (same as zip file location)
#     extract_folder = os.path.dirname(zip_path)

#     # Extract the zip file
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_folder)
#         print(f"Extracted all files in {zip_path} to {extract_folder}")

# # Example usage
# zip_path = "/content/drive/MyDrive/Skincare Recommendation Dataset 2/Sephora Products and Skincare Reviews.zip"
# extract_zip_from_drive(zip_path)


In [ ]:
import pandas as pd

def load_dataset(file_path):
    try:
        # Read the file into a Pandas DataFrame
        df = pd.read_csv(file_path, index_col = 0, dtype={'author_id':'str'})
        return df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None


dataset_paths = [
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/product_info.csv",
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/reviews_0_250.csv",
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/reviews_250_500.csv",
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/reviews_500_750.csv",
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/reviews_750_1000.csv",
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/reviews_1000_1500.csv",
    "/content/drive/MyDrive/Skincare Recommendation Dataset 2/reviews_1500_end.csv"
]

# Load each dataset into separate variables
product_info_df = load_dataset(dataset_paths[0])
reviews_0_250 = load_dataset(dataset_paths[1])
reviews_250_500 = load_dataset(dataset_paths[2])
reviews_500_750 = load_dataset(dataset_paths[3])
reviews_750_1000 = load_dataset(dataset_paths[4])
reviews_1000_1500 = load_dataset(dataset_paths[5])
reviews_1500_end = load_dataset(dataset_paths[6])

# Concatenate the reviews datasets
reviews_combined_df = pd.concat([reviews_0_250, reviews_250_500, reviews_500_750, reviews_750_1000, reviews_1000_1500, reviews_1500_end], ignore_index=True)

In [ ]:
reviews_combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1301136 entries, 0 to 1301135
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   author_id                 1301136 non-null  object 
 1   rating                    1301136 non-null  int64  
 2   is_recommended            1107162 non-null  float64
 3   helpfulness               631670 non-null   float64
 4   total_feedback_count      1301136 non-null  int64  
 5   total_neg_feedback_count  1301136 non-null  int64  
 6   total_pos_feedback_count  1301136 non-null  int64  
 7   submission_time           1301136 non-null  object 
 8   review_text               1299520 non-null  object 
 9   review_title              930754 non-null   object 
 10  skin_tone                 1103798 non-null  object 
 11  eye_color                 1057734 non-null  object 
 12  skin_type                 1172830 non-null  object 
 13  hair_color                1

In [ ]:
product_info_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8494 entries, P473671 to P505461
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_name        8494 non-null   object 
 1   brand_id            8494 non-null   int64  
 2   brand_name          8494 non-null   object 
 3   loves_count         8494 non-null   int64  
 4   rating              8216 non-null   float64
 5   reviews             8216 non-null   float64
 6   size                6863 non-null   object 
 7   variation_type      7050 non-null   object 
 8   variation_value     6896 non-null   object 
 9   variation_desc      1250 non-null   object 
 10  ingredients         7549 non-null   object 
 11  price_usd           8494 non-null   float64
 12  value_price_usd     451 non-null    float64
 13  sale_price_usd      270 non-null    float64
 14  limited_edition     8494 non-null   int64  
 15  new                 8494 non-null   int64  
 16  on

In [ ]:
product_info_df.shape

(8494, 26)

In [ ]:
reviews_combined_df.shape

(1301136, 18)

In [ ]:
product_info_df.columns

Index(['product_name', 'brand_id', 'brand_name', 'loves_count', 'rating',
       'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

In [ ]:
product_info_df.columns

Index(['product_name', 'brand_id', 'brand_name', 'loves_count', 'rating',
       'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')

In [ ]:
product_info_df['primary_category'].value_counts()

primary_category
Skincare           2420
Makeup             2369
Hair               1464
Fragrance          1432
Bath & Body         405
Mini Size           288
Men                  60
Tools & Brushes      52
Gifts                 4
Name: count, dtype: int64

In [ ]:
product_skincare = product_info_df[product_info_df['primary_category'] == 'Skincare']

In [ ]:
product_skincare.shape

(2420, 26)

In [ ]:
product_skincare['secondary_category'].value_counts()

secondary_category
Moisturizers              551
Treatments                466
Cleansers                 361
Value & Gift Sets         196
Eye Care                  186
Masks                     166
Mini Size                 112
Sunscreen                 108
High Tech Tools            80
Wellness                   79
Lip Balms & Treatments     61
Self Tanners               53
Shop by Concern             1
Name: count, dtype: int64

In [ ]:
reviews_combined_df['product_name'].value_counts()

product_name
Lip Sleeping Mask Intense Hydration with Vitamin C        16138
Soy Hydrating Gentle Face Cleanser                         8736
100 percent Pure Argan Oil                                 7763
Ultra Repair Cream Intense Hydration                       7547
Alpha Beta Extra Strength Daily Peel Pads                  7414
                                                          ...  
Major Eye Impact Repair + Brighten Skincare Set               1
Clarifying Peel Pads Purify + Exfoliate                       1
Smooth + FAB Travel Size Duo                                  1
The Glow Around 4-Piece Travel Skincare Essentials Set        1
Gentle Hydra-Gel Face Cleanser                                1
Name: count, Length: 2334, dtype: int64

In [ ]:
import pandas as pd

# Assume reviews_combined_df is already defined

# Calculate the count of each product_name
product_counts = reviews_combined_df['product_name'].value_counts()

# Find products with counts greater than 500
popular_products = product_counts[product_counts > 500].index

# Filter the DataFrame to only include rows with these popular products
reviews_filtered_df = reviews_combined_df[reviews_combined_df['product_name'].isin(popular_products)]

# Show the filtered DataFrame
reviews_filtered_df.head(5)


,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
5,42802569154,4,1.0,1.0,1,0,1,2023-03-19,The scent isn’t my favourite but it works grea...,Great!,lightMedium,brown,normal,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [ ]:
reviews_filtered_df['product_name'].value_counts()

product_name
Lip Sleeping Mask Intense Hydration with Vitamin C    16138
Soy Hydrating Gentle Face Cleanser                     8736
100 percent Pure Argan Oil                             7763
Ultra Repair Cream Intense Hydration                   7547
Alpha Beta Extra Strength Daily Peel Pads              7414
                                                      ...  
Vinosource-Hydra SOS Intense Hydration Moisturizer      502
Self-Tan Luxe Whipped Crème Mousse                      502
Glow Stick Sunscreen SPF 50 PA++++                      502
Ascorbic Acid 8% + Alpha Arbutin 2%                     501
Eye Doctor Moisture Care For Skin Around Eyes           501
Name: count, Length: 573, dtype: int64

In [ ]:
reviews_filtered_df['brand_name'].value_counts()

brand_name
CLINIQUE                             50527
Tatcha                               49337
fresh                                47545
Drunk Elephant                       46262
Glow Recipe                          38495
                                     ...  
Paula's Choice                        1200
WASO                                  1046
Dr. Lara Devgan Scientific Beauty      835
Dr. Zenovia Skincare                   519
Moon Juice                             510
Name: count, Length: 79, dtype: int64

In [ ]:
product_skincare.isna().sum()

product_name             0
brand_id                 0
brand_name               0
loves_count              0
rating                  69
reviews                 69
size                   365
variation_type         372
variation_value        420
variation_desc        2407
ingredients            134
price_usd                0
value_price_usd       2258
sale_price_usd        2375
limited_edition          0
new                      0
online_only              0
out_of_stock             0
sephora_exclusive        0
highlights             417
primary_category         0
secondary_category       0
tertiary_category      380
child_count              0
child_max_price       2018
child_min_price       2018
dtype: int64

In [ ]:
reviews_filtered_df.isna().sum()

author_id                        0
rating                           0
is_recommended              174295
helpfulness                 567743
total_feedback_count             0
total_neg_feedback_count         0
total_pos_feedback_count         0
submission_time                  0
review_text                   1401
review_title                298853
skin_tone                   165563
eye_color                   212473
skin_type                   111845
hair_color                  221443
product_id                       0
product_name                     0
brand_name                       0
price_usd                        0
dtype: int64

In [ ]:
# Normalize case by converting 'product_name' and 'brand_name' to lowercase for both DataFrames
product_skincare['product_name'] = product_skincare['product_name'].str.lower()
product_skincare['brand_name'] = product_skincare['brand_name'].str.lower()

reviews_filtered_df['product_name'] = reviews_filtered_df['product_name'].str.lower()
reviews_filtered_df['brand_name'] = reviews_filtered_df['brand_name'].str.lower()



In [ ]:
# Inner Merge for Matrix Factorization
mf_merged_df = pd.merge(reviews_filtered_df, product_skincare,
                        on=['product_name', 'brand_name'],
                        how='inner')

In [ ]:
mf_df = mf_merged_df

In [ ]:
mf_df.isna().sum()

author_id                         0
rating_x                          0
is_recommended               179084
helpfulness                  574608
total_feedback_count              0
total_neg_feedback_count          0
total_pos_feedback_count          0
submission_time                   0
review_text                    1420
review_title                 302303
skin_tone                    168260
eye_color                    217777
skin_type                    113722
hair_color                   226965
product_id                        0
product_name                      0
brand_name                        0
price_usd_x                       0
brand_id                          0
loves_count                       0
rating_y                          0
reviews                           0
size                          31042
variation_type                43667
variation_value               50256
variation_desc              1061642
ingredients                   15414
price_usd_y                 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

class Missing_Values_Preprocessing:
    def calculate_missing_values_percent(self, df):
        """Calculate the percentage of missing values for each column in the DataFrame."""
        missing_percent = df.isnull().sum() * 100 / len(df)
        missing_percent_df = pd.DataFrame({'column_name': df.columns,
                                           'percent_missing': missing_percent}).reset_index(drop=True)
        return missing_percent_df

    def drop_columns_with_missing_values(self, df, threshold=20):
        """
        Drop columns from the DataFrame if they have more than a specified percentage of missing values.
        """
        percent_missing = df.isna().sum() * 100 / len(df)
        columns_to_drop = percent_missing[percent_missing > threshold].index
        df_dropped = df.drop(columns=columns_to_drop)
        print(f"Dropped columns: {list(columns_to_drop)}")
        return df_dropped

    def drop_rows_with_missing_values(self, df):
        """
        Drop rows from the DataFrame that have any missing values.
        """
        initial_row_count = len(df)
        df_dropped = df.dropna()
        final_row_count = len(df_dropped)
        print(f"Dropped rows: {initial_row_count - final_row_count}")
        return df_dropped

    def check_distribution_of_missing(self, df):
        """
        Check the distribution of columns with missing values to determine if they are normally distributed or skewed.
        """
        columns_with_missing = df.columns[df.isnull().any() & (df.dtypes != "object")].tolist()
        skewness = df[columns_with_missing].apply(lambda x: skew(x.dropna()), axis=0)
        distribution_type = skewness.apply(lambda x: 'Highly skewed' if abs(x) > 1 else
                                                      'Moderately skewed' if abs(x) > 0.5 else
                                                      'Approximately symmetric')
        distribution_df = pd.DataFrame({'column_name': columns_with_missing,
                                        'skewness': skewness,
                                        'distribution': distribution_type}).reset_index(drop=True)
        return distribution_df

    def visualize_distribution(self, df, column_name):
        """
        Visualize the distribution of a given column using histogram and density plot.
        """
        if column_name in df.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(df[column_name], kde=True, stat="density", linewidth=0)
            plt.title(f'Distribution of {column_name}')
            plt.xlabel(column_name)
            plt.ylabel('Density')
            plt.show()
        else:
            print(f"Column '{column_name}' does not exist in the DataFrame.")

    def check_for_duplicates(self, df):
        """
        Check for duplicate rows in the DataFrame.

        Returns:
        - The number of duplicate rows found.
        """
        duplicate_rows = df.duplicated().sum()
        print(f"Number of duplicate rows: {duplicate_rows}")
        return duplicate_rows

    def drop_duplicates(self, df):
        """
        Drop duplicate rows from the DataFrame.

        Returns:
        - DataFrame with duplicates removed.
        """

        initial_row_count = len(df)
        df_dropped = df.drop_duplicates()
        final_row_count = len(df_dropped)
        print(f"Dropped duplicate rows: {initial_row_count - final_row_count}")
        return df_dropped


# Assuming mf_df is your DataFrame after the inner merge
preprocessor = Missing_Values_Preprocessing()

In [ ]:
mf_df2 = preprocessor.drop_columns_with_missing_values(mf_df)

Dropped columns: ['helpfulness', 'review_title', 'eye_color', 'hair_color', 'variation_desc', 'value_price_usd', 'sale_price_usd', 'child_max_price', 'child_min_price']


In [ ]:
mf_df2.isna().sum()

author_id                        0
rating_x                         0
is_recommended              179084
total_feedback_count             0
total_neg_feedback_count         0
total_pos_feedback_count         0
submission_time                  0
review_text                   1420
skin_tone                   168260
skin_type                   113722
product_id                       0
product_name                     0
brand_name                       0
price_usd_x                      0
brand_id                         0
loves_count                      0
rating_y                         0
reviews                          0
size                         31042
variation_type               43667
variation_value              50256
ingredients                  15414
price_usd_y                      0
limited_edition                  0
new                              0
online_only                      0
out_of_stock                     0
sephora_exclusive                0
highlights          

In [ ]:
# Columns to drop
columns_to_drop = [
    'child_count', 'tertiary_category', 'secondary_category', 'primary_category',
    'highlights', 'sephora_exclusive', 'out_of_stock', 'online_only', 'new',
    'limited_edition', 'variation_type', 'variation_value', 'size', 'brand_id',
    'submission_time'
]

# Drop the specified columns
mf_df3 = mf_df2.drop(columns=columns_to_drop, errors='ignore')




In [ ]:
mf_df3.isna().sum()

author_id                        0
rating_x                         0
is_recommended              179084
total_feedback_count             0
total_neg_feedback_count         0
total_pos_feedback_count         0
review_text                   1420
skin_tone                   168260
skin_type                   113722
product_id                       0
product_name                     0
brand_name                       0
price_usd_x                      0
loves_count                      0
rating_y                         0
reviews                          0
ingredients                  15414
price_usd_y                      0
dtype: int64

In [ ]:
mf_df4 = mf_df3.dropna()

In [ ]:
mf_df4.isna().sum()

author_id                   0
rating_x                    0
is_recommended              0
total_feedback_count        0
total_neg_feedback_count    0
total_pos_feedback_count    0
review_text                 0
skin_tone                   0
skin_type                   0
product_id                  0
product_name                0
brand_name                  0
price_usd_x                 0
loves_count                 0
rating_y                    0
reviews                     0
ingredients                 0
price_usd_y                 0
dtype: int64

In [ ]:
# Dictionary to rename columns with _x and _y suffixes
rename_columns = {
    'rating_x': 'rating_reviews',
    'rating_y': 'rating_products',
    'price_usd_x': 'price_usd_reviews',
    'price_usd_y': 'price_usd_products'
}

# Rename the columns in the DataFrame
mf_df4.rename(columns=rename_columns, inplace=True)




In [ ]:
mf_df4[['rating_reviews', 'rating_products']]

,rating_reviews,rating_products
1,5,4.3508
3,5,4.3508
4,4,4.3508
5,2,4.3508
6,5,4.3508
...,...,...
1069777,5,4.7850
1069778,5,4.7850
1069779,5,4.7850
1069780,5,4.7850


In [ ]:
mf_df4['author_id'].value_counts().iloc[7000:]

author_id
11652795365    10
22633803755    10
5552996259     10
23576521763    10
22201794098    10
               ..
39891258255     1
30088025069     1
36306363787     1
6286897153      1
26685757939     1
Name: count, Length: 320817, dtype: int64

In [ ]:
# Calculate the count of each author_id
author_counts = mf_df4['author_id'].value_counts()

# Filter author_ids with more than 10 occurrences
authors_over_10 = author_counts[author_counts > 9].index

# Create the new DataFrame including only rows with author_ids that have more than 10 occurrences
mf_df5 = mf_df4[mf_df4['author_id'].isin(authors_over_10)]

In [ ]:
mf_df5['author_id'].value_counts()

author_id
1696370280    242
1288462295    138
1930716686    128
2330399812     96
5060164185     95
             ... 
5401594584     10
1591112910     10
5713635898     10
2411631269     10
7178152696     10
Name: count, Length: 8681, dtype: int64

In [ ]:
mf_df5.shape

(143669, 18)

In [ ]:
mf_df5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143669 entries, 28 to 1069715
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   author_id                 143669 non-null  object 
 1   rating_reviews            143669 non-null  int64  
 2   is_recommended            143669 non-null  float64
 3   total_feedback_count      143669 non-null  int64  
 4   total_neg_feedback_count  143669 non-null  int64  
 5   total_pos_feedback_count  143669 non-null  int64  
 6   review_text               143669 non-null  object 
 7   skin_tone                 143669 non-null  object 
 8   skin_type                 143669 non-null  object 
 9   product_id                143669 non-null  object 
 10  product_name              143669 non-null  object 
 11  brand_name                143669 non-null  object 
 12  price_usd_reviews         143669 non-null  float64
 13  loves_count               143669 non-null  int6

In [ ]:
mf_df5['product_id'].value_counts()

product_id
P479841    1610
P469538    1402
P476729    1402
P505346    1390
P455236    1370
           ... 
P459127       7
P459130       3
P503872       2
P94812        2
P7365         1
Name: count, Length: 570, dtype: int64

In [ ]:
mf_df5['product_name'].nunique()

564

In [ ]:
import os

#To ensure the directory exists
os.makedirs('/content/drive/MyDrive/Skincare Recommendation Dataset 2/Cleaned_Products_And_Reviews_Dataset', exist_ok=True)

# Path to save the CSV
csv_path = '/content/drive/MyDrive/Skincare Recommendation Dataset 2/Cleaned_Products_And_Reviews_Dataset/cleaned_products_and_reviews_dataset.csv'

# Save the DataFrame to CSV
mf_df5.to_csv(csv_path, index=False)  # Set index=False to avoid saving the index as an additional column

print(f"CSV saved successfully at {csv_path}")


CSV saved successfully at /content/drive/MyDrive/Skincare Recommendation Dataset 2/Cleaned_Products_And_Reviews_Dataset/cleaned_products_and_reviews_dataset.csv


In [ ]:
pip install scikit-surprise

In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load data
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(collab_filtering_df[['author_id', 'product_id', 'rating_reviews']], reader)

# Split data into training and test set
trainset, testset = train_test_split(data, test_size=0.25)

# Train the SVD algorithm
algo = SVD()
algo.fit(trainset)


In [ ]:
# Make predictions on the test set
predictions = algo.test(testset)
v
# Compute and print Root Mean Squared Error
rmse = accuracy.rmse(predictions)

RMSE: 0.7252


In [ ]:
import pandas as pd

# Function to predict ratings for all products for a specific user and return a DataFrame
def predict_all_ratings(user_id, product_pool, algo):
    # Predict ratings for all products in the product pool for the given user
    predictions = [algo.predict(user_id, prod) for prod in product_pool]

    # Create a DataFrame from the predictions
    predictions_df = pd.DataFrame([(pred.uid, pred.iid, pred.est) for pred in predictions],
                                  columns=['User ID', 'Product ID', 'Estimated Rating'])

    # Join with the product names for more readable output
    product_names = mf_df5[['product_id', 'product_name']].drop_duplicates().set_index('product_id')
    predictions_df = predictions_df.set_index('Product ID').join(product_names, how='left').reset_index()
    predictions_df.rename(columns={'index': 'Product ID', 'product_name': 'Product Name'}, inplace=True)

    return predictions_df

product_pool = collab_filtering_df['product_id'].unique()

# Predict ratings for all products for a specific user
user_id = '1238130325'  # A random user id
all_ratings_df = predict_all_ratings(user_id, product_pool, algo)

In [ ]:
all_ratings_df

,Product ID,User ID,Estimated Rating,Product Name
0,P420652,1238130325,4.590868,lip sleeping mask intense hydration with vitam...
1,P7880,1238130325,4.691383,soy hydrating gentle face cleanser
2,P218700,1238130325,4.859852,100 percent pure argan oil
3,P248407,1238130325,5.000000,ultra repair cream intense hydration
4,P269122,1238130325,4.919732,alpha beta extra strength daily peel pads
...,...,...,...,...
565,P442754,1238130325,4.493114,ascorbic acid 8% + alpha arbutin 2%
566,P429953,1238130325,4.319110,glow stick sunscreen spf 50 pa++++
567,P471101,1238130325,5.000000,resurface+ aha/bha niacinamide exfoliating pad...
568,P297531,1238130325,4.666268,eye doctor moisture care for skin around eyes


In [ ]:
all_ratings_df.sort_values(by='Estimated Rating', ascending=False).head(20)

,Product ID,User ID,Estimated Rating,Product Name
285,P482535,1238130325,5.0,strawberry smooth bha + aha salicylic acid serum
396,P501254,1238130325,5.0,watermelon glow aha night treatment
391,P475543,1238130325,5.0,guava vitamin c dark spot treatment serum
203,P482739,1238130325,5.0,urban environment oil-free sunscreen broad-spe...
202,P454018,1238130325,5.0,the serum stick: treatment & touch up balm
201,P415747,1238130325,5.0,brazilian kiss cupuaçu lip butter
393,P467615,1238130325,5.0,cicapair tiger grass sleepair intensive mask
198,P474938,1238130325,5.0,vinopure natural salicylic acid pore minimizin...
196,P411539,1238130325,5.0,cicapair tiger grass cream
536,P483076,1238130325,5.0,c.e.o. afterglow brightening vitamin c moistur...


In [ ]:
import pickle

# Save the algorithm to a file
file_name = '/content/drive/MyDrive/Skincare Recommendation Dataset 2/Cleaned_Products_And_Reviews_Dataset/svd_collab_filter_model.pkl'

with open(file_name, 'wb') as f:
    pickle.dump(algo, f)

print(f"Model saved successfully at {file_name}")


Model saved successfully at /content/drive/MyDrive/Skincare Recommendation Dataset 2/Cleaned_Products_And_Reviews_Dataset/svd_collab_filter_model.pkl
